In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn import preprocessing

In [2]:
users = pd.read_pickle('Users_2.pkl')

In [3]:
# Cargar Modelo
model = CatBoostClassifier().load_model("modelo")

In [4]:
usuarios = users[['id','screen_name','complete_name']]

In [5]:
users.crawled_at = pd.to_datetime(users.crawled_at).dt.floor('S') # Corregir formato timestamp
users.created_at = pd.to_datetime(users.created_at).dt.floor('S') # Corregir formato timestamp
users.description = users.description.notna()  # Convertir a Dummy
users.screen_name = users.screen_name.notna()  # Convertir a Dummy
users.location = users.location.notna()  # Convertir a Dummy

In [6]:
# Favourites Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'favs_tiempo'] = users.favs_count.astype(int) / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='favs_count',inplace=True)

In [7]:
# Statuses Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'statuses_tiempo'] = users.tweet_count.astype(int) / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='tweet_count',inplace=True)

In [8]:
# Followers Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'followers_tiempo'] = users.followers_count.astype(int) / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='followers_count',inplace=True)

In [9]:
# Friends Count = The number of Tweets this user has liked in the account’s lifetime
# Transformar en ratio sobre tiempo de existencia
users.loc[:,'friends_tiempo'] = users.following_count.astype(int) / ((pd.to_datetime(users.crawled_at,errors='coerce').astype(int) -  pd.to_datetime(users.created_at,errors='coerce').astype(int))/10000000000000000)
# Eliminar columna original
users.drop(columns='following_count',inplace=True)

In [10]:
# Quedarnos con las columnas necesarias
users = users[['screen_name', 'description', 'location', 'listed_count', 'statuses_tiempo', 'favs_tiempo','followers_tiempo', 'friends_tiempo']]
# Preparar datos para aplicar el modelo
atributos = users.columns
to_predict = preprocessing.scale(users, axis = 0)

In [11]:
# Predecir Probabilidades
predicciones = model.predict_proba(to_predict)

In [12]:
# Joinear con nombres de usuarios, limpiar indices y joinear por orden de aparicion
usuarios_prob = pd.DataFrame(usuarios.screen_name.to_list()).join(pd.DataFrame(predicciones)[1])
usuarios_prob.columns = ['Usuario','Prob_Bot']
# Quedarnos con los que dan probabilidad mayor al 99% de ser bots segun el modelo basico
usuarios_prob = usuarios_prob[usuarios_prob['Prob_Bot']>0.99]

In [15]:
# Lista Usuarios
usuarios_prob.sort_values(by='Prob_Bot',ascending=False)

,Usuario,Prob_Bot
316,likessssok,0.999969
3988,iFutboler0,0.999968
3178,Registraduria,0.999963
2093,carlosvecchio,0.999963
2226,marcelobarti,0.999962
2086,GerardoCabreraR,0.999953
1644,HambreCero,0.999952
3948,marquina04,0.999949
170,BaironValleP,0.999948
3381,ilustremos,0.999947
